In [ ]:
# Script to generate uncorrelated models for a mechanism

In [ ]:
# note, avoids deepcopy due to some issues copying

In [1]:
import os
import sys
import copy
import pickle
import subprocess

import numpy as np
import rmgpy.chemkin

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the model and the saved uncertainty covariance matrix.
# Only looks at uncorrelated parameters, so you can fill the
# diagonal of a zero matrix and get the same results as using
# the full covariance matrix

basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/rmg_model'
model_dir = '/scratch/harris.se/autoscience/correlated/models'  # where the output cti's go

base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
# RMG_cti_path = os.path.join(basedir, 'chem_annotated.cti')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
# RMG_gas = ct.Solution(RMG_cti_path)


covariance_file = '/work/westgroup/harris.se/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)


In [3]:
uniform_randoms = np.load('randoms1001.npy')

## Generate the correlated random uniform variables

In [ ]:
# Generate the random numbers
np.random.seed(400)
mean_vec = np.zeros(Sigma_k.shape[0])  # Sigma_k is a square matrix
N = 5000
normal_randoms = np.random.multivariate_normal(mean_vec, Sigma_k, size=N)

In [ ]:
# compute the CDF using the first 1001 of the random samples
sorted_normals = np.sort(normal_randoms, axis=0)
CDF = np.zeros((1001, sorted_normals.shape[1]))
for rxn_index in range(0, sorted_normals.shape[1]):
    # the variance represents the 1/12*(b-a)^2
    # the variance represents the a^2/3
    var = Sigma_k[rxn_index, rxn_index]
    a = np.sqrt(3 * var)  # assumed uniform distribution is symmetric about zero
    CDF[:, rxn_index] = np.linspace(-a, a, CDF.shape[0])

In [ ]:
# function to actually convert the normal randoms into uniform randoms
def normal_to_uniform(sample, rxn_index):
    # assumes CDF is already defined
    # find the closest value to the input
    distance = np.abs(CDF[:, rxn_index] - sample)
    closest_index = np.argmin(distance)
    return CDF[closest_index, rxn_index]

In [ ]:
# convert all those normal variables to uniforms
uniform_randoms = np.zeros(normal_randoms.shape)
for i in range(0, normal_randoms.shape[0]):
    for j in range(0, normal_randoms.shape[1]):
        uniform_randoms[i, j] = normal_to_uniform(normal_randoms[i, j], j)
np.save('uniform_randoms.npy', uniform_randoms)

In [ ]:
# check correlation coefficients with a few examples
# reaction 0 is independent, then 324 is correlated with 325, and 327
print(np.corrcoef(uniform_randoms[:, 324], uniform_randoms[:, 325]))
print()
print(np.corrcoef(uniform_randoms[:, 324], uniform_randoms[:, 0]))

In [4]:
def perturb_reaction(rxn, delta):
    # takes in an RMG reaction object
    # delta is the ln(k) amount to perturb the A factor
    # delta is a multiplicative factor- units don't matter, yay!
    # does not deepycopy because there's some issues with rmgpy.reactions copying

    rxn.kinetics.A.value *= np.exp(delta)

#     return rxn

In [ ]:
def plot_reaction_perturbation(original_reaction, perturbed_reaction, uncertainty):
    # function to show how the reaction kinetics have been changed
    
    plt.xlabel('1000 / T (K^-1)')
    plt.ylabel('ln(k)')

    T = np.linspace(300, 3000, 1001)

    k_orig = np.zeros(len(T))
    k_perturb = np.zeros(len(T))
    for i in range(0, len(T)):
        k_orig[i] = original_reaction.kinetics.get_rate_coefficient(T[i], 101325)
        k_perturb[i] = perturbed_reaction.kinetics.get_rate_coefficient(T[i], 101325)
    plt.plot(1000.0 / T, np.log(k_orig))
    plt.plot(1000.0 / T, np.log(k_perturb))
    
    plt.plot(1000.0 / T, np.log(k_orig) + uncertainty, color='grey')
    plt.plot(1000.0 / T, np.log(k_orig) - uncertainty, color='grey')

    plt.legend(['Original', 'Perturbed'])
    plt.show()

In [ ]:

rxn = copy.deepcopy(reaction_list[324])
rxn.family

perturb_reaction(reaction_list[324], 1.0)


plot_reaction_perturbation(rxn, reaction_list[324], 2.0)

### Generate Random Uniform Perturbations for A Factor

Bounds are determined by variance
https://en.wikipedia.org/wiki/Continuous_uniform_distribution

$var = \frac{1}{12}\frac{1}{(b-a)^2}$

We're constructing our perturbation centered around zero, so $b=-a$

$var = \frac{1}{12}\frac{1}{(-2a)^2} = \frac{1}{12}\frac{1}{4a^2}$


$a^2 = \frac{1}{48 var}$

$a = \frac{1}{4\sqrt{3 var}}$

In [ ]:
print(uniform_randoms.shape)

In [ ]:
uniform_randoms[5, 324]

In [5]:
reaction_list[324].kinetics

Arrhenius(A=(474158,'s^-1'), n=1.155, Ea=(16.101,'kcal/mol'), T0=(1,'K'), comment="""Estimated using template [R7H_OOCs4;O_rad_out;XH_out] for rate rule [R7H_OOCs4;O_rad_out;O_H_out]
Euclidian distance = 1.0
family: intra_H_migration""")

In [8]:
# for j in range(0, N):
for k in range(0, 10):
    print(k)
    # load a fresh mechanism
    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)

    # perturb all of the reactions
    for i in range(0, len(reaction_list)):
#         if type(reaction_list[i].kinetics) == rmgpy.kinetics.arrhenius.MultiArrhenius:
#             continue
#         if type(reaction_list[i].kinetics) == rmgpy.kinetics.falloff.ThirdBody:
#             continue
#         if type(reaction_list[i].kinetics) == rmgpy.kinetics.falloff.Troe:
#             continue
#         if type(reaction_list[i].kinetics) == rmgpy.kinetics.arrhenius.PDepArrhenius:
#             continue
#         if type(reaction_list[i].kinetics) == rmgpy.kinetics.arrhenius.MultiPDepArrhenius:
#             continue
#         if type(reaction_list[i].kinetics) == rmgpy.kinetics.falloff.Lindemann:
#             continue
#         if type(reaction_list[i].kinetics) == rmgpy.kinetics.chebyshev.Chebyshev:
#             continue
        try:

            delta = uniform_randoms[k, i]
            perturb_reaction(reaction_list[i], delta)
        except AttributeError:
            continue
        
    # save the new mechanism
    chemkin_file = os.path.join(model_dir, f'chem_{k:04}.inp')
    rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
    subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={model_dir}/chem_{k:04}.cti'])
    os.remove(chemkin_file)

0


Wrote CTI mechanism file to '/scratch/harris.se/autoscience/correlated/models/chem_0000.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
1


Wrote CTI mechanism file to '/scratch/harris.se/autoscience/correlated/models/chem_0001.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
2


Wrote CTI mechanism file to '/scratch/harris.se/autoscience/correlated/models/chem_0002.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
3


Wrote CTI mechanism file to '/scratch/harris.se/autoscience/correlated/models/chem_0003.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
4


...

Wrote CTI mechanism file to '/scratch/harris.se/autoscience/correlated/models/chem_0005.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
6


Wrote CTI mechanism file to '/scratch/harris.se/autoscience/correlated/models/chem_0006.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
7


Wrote CTI mechanism file to '/scratch/harris.se/autoscience/correlated/models/chem_0007.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
8


Wrote CTI mechanism file to '/scratch/harris.se/autoscience/correlated/models/chem_0008.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.
9


Wrote CTI mechanism file to '/scratch/harris.se/autoscience/correlated/models/chem_0009.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.


In [ ]:



#     plot_reaction_perturbation(reaction_list[i], new_rxn, Sigma_k[i, i])

In [ ]:
print(reaction_list[288])
print(reaction_list[288].kinetics)

rxn288 = copy.copy(reaction_list[288])
rxn288_2 = copy.copy(reaction_list[288])
perturb_reaction(rxn288_2, -6)

plot_reaction_perturbation(rxn288, rxn288_2, 1.0)

print(rxn288_2.kinetics)